In [1]:
from keras.models import Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
import numpy as np
import os
import cv2
from utils import decode_netout, compute_overlap, compute_ap
from keras.applications.mobilenet import MobileNet
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from preprocessing import BatchGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from backend import TinyYoloFeature, FullYoloFeature, MobileNetFeature, SqueezeNetFeature, Inception3Feature, VGG16Feature, ResNet50Feature

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K
import tensorflow as tf
K.set_learning_phase(1) 

sess = tf.InteractiveSession()
K.set_session(sess)

In [3]:
input_size = 224
feature_extractor = MobileNetFeature(input_size)
nb_box = 5
nb_class = 3
grid_h, grid_w = feature_extractor.get_output_shape()
max_box_per_image = 10
sequence_length = 8

conv_layer = Conv2D(nb_box * (4 + 1 + nb_class), 
    (1,1), strides=(1,1), 
    padding='same', 
    name='DetectionLayer', 
    kernel_initializer='lecun_normal')

In [4]:
from keras.layers import TimeDistributed, Flatten, LSTM
from keras.models import Sequential
from keras.losses import categorical_crossentropy

In [5]:
from ttlstm import TTLSTMCell

In [6]:
input_video = Input(shape=(sequence_length, input_size, input_size, 3))
true_boxes = Input(shape=(sequence_length, 1, 1, 1, max_box_per_image , 4))

features = TimeDistributed(feature_extractor.feature_extractor)(input_video)

detection = TimeDistributed(conv_layer)(features)
detection = TimeDistributed(Flatten())(detection)
#detection = LSTM(128, return_sequences=True)(detection)
#detection = LSTM(grid_h * grid_w * nb_box *  (4 + 1 + nb_class), return_sequences=True)(detection)
#detection = Reshape((sequence_length, grid_h, grid_w, nb_box, 4 + 1 + nb_class))(detection)

tt_input_shape = [7, 20, 10, 7]
tt_output_shape = tt_input_shape
tt_ranks = [1, 4, 4, 4, 1]



detection = TTLSTMCell(1960, [tt_input_shape, tt_input_shape])(detection)

# detection = trnn.TensorLSTMCell(tt_input_shape=tt_input_shape, tt_output_shape=tt_output_shape,
#                    tt_ranks=tt_ranks,
#                     return_sequences=True,
#                     dropout=0.25, recurrent_dropout=0.25, activation='tanh')(detection)

output = detection

model = Model(input_video, output)

ValueError: Scope reuse is False and initializer is not provided.

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 8, 224, 224, 3)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 8, 7, 7, 1024)     3228864   
_________________________________________________________________
time_distributed_14 (TimeDis (None, 8, 7, 7, 40)       41000     
_________________________________________________________________
time_distributed_15 (TimeDis (None, 8, 1960)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 8, 1960)           30740640  
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 7, 7, 5, 8)     0         
Total params: 34,010,504
Trainable params: 33,988,616
Non-trainable params: 21,888
___________________________________________________________

In [4]:
np.prod([7, 4, 10, 7])

1960